(sec:pandas10min_2)=
# 판다스 10분 완성 2부


**필수 라이브러리**

In [1]:
import numpy as np
import pandas as pd

## 합병과 결합: merge-join-concat

- 참고: [Merging section](https://pandas.pydata.org/docs/user_guide/merging.html#merging)

### 종/횡 결합: `pd.concat()` 함수

`pd.concat()` 함수는 여러 개의 데이터프레임을 하나로 합친다.

- `axis=0`: 종 결합. 즉 데이터프레임 여러 개의 위아래 결합.

In [2]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)

df1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [3]:
df2 = pd.DataFrame(
    {
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"],
    },
    index=[4, 5, 6, 7],
)

df2

,A,B,C,D
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


In [4]:
df3 = pd.DataFrame(
    {
        "A": ["A8", "A9", "A10", "A11"],
        "B": ["B8", "B9", "B10", "B11"],
        "C": ["C8", "C9", "C10", "C11"],
        "D": ["D8", "D9", "D10", "D11"],
    },
    index=[8, 9, 10, 11],
)

df3

,A,B,C,D
8,A8,B8,C8,D8
9,A9,B9,C9,D9
10,A10,B10,C10,D10
11,A11,B11,C11,D11


In [5]:
pd.concat([df1, df2, df3]) # axis=0 이 기본값

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7
8,A8,B8,C8,D8
9,A9,B9,C9,D9


- `axis=1`: 횡 결합. 즉 데이터프레임 여러 개의 좌우 결합.

In [6]:
df4 = pd.DataFrame(
    {
        "B": ["B2", "B3", "B6", "B7"],
        "D": ["D2", "D3", "D6", "D7"],
        "F": ["F2", "F3", "F6", "F7"],
    },
    index=[2, 3, 6, 7],
)

df4

,B,D,F
2,B2,D2,F2
3,B3,D3,F3
6,B6,D6,F6
7,B7,D7,F7


In [7]:
pd.concat([df1, df4], axis=1)

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3
6,NaN,NaN,NaN,NaN,B6,D6,F6
7,NaN,NaN,NaN,NaN,B7,D7,F7


인덱스를 기존의 데이터프레임과 통일시키기 위해 리인덱싱을 활용할 수도 있다.

In [8]:
df1.index

Int64Index([0, 1, 2, 3], dtype='int64')

In [9]:
pd.concat([df1, df4], axis=1).reindex(df1.index)

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3


In [10]:
pd.concat([df1, df4.reindex(df1.index)], axis=1)

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3


### 합병: `pd.merge()` 함수

`pd.merge()` 함수 는 SQL 방식으로 특정 열을 기준으로 두 개의 데이터프레임을 합친다.
다양한 옵션을 지원하는 매우 강력한 도구이다.

- 참고: [Database style joining](https://pandas.pydata.org/docs/user_guide/merging.html#merging-join)

**예제**

실습을 위해 아래 두 데이터프레임을 이용한다.

In [11]:
left = pd.DataFrame({"key": ["foo", "foo"], "lval": [1, 2]})
right = pd.DataFrame({"key": ["foo", "foo"], "rval": [4, 5]})

In [12]:
left

,key,lval
0,foo,1
1,foo,2


In [13]:
right

,key,rval
0,foo,4
1,foo,5


- `on="key"` 키워드 인자
    - `key` 열에 사용된 항목 각각에 대해 다른 열에서 해당 항목과 연관된 값들을 조합할 수 있는 모든 경우의 수를 다룬다.
    - `foo` 값에 대해 `lval` 열에서 2개의 값이,
        `rval` 열에서 2개의 값이 있기에 `foo`와 관련해서 총 4개의 경우가 생성된다.
    <br><br>
    
    | `key` | `left.lval` | `right.rval` | 경우의 수 |
    | :---: | :---: | :---: | :---: |
    | `foo` | `1, 2` | `4, 5` | 4 |

In [14]:
pd.merge(left, right, on="key")

,key,lval,rval
0,foo,1,4
1,foo,1,5
2,foo,2,4
3,foo,2,5


**예제**

In [15]:
left = pd.DataFrame({"key": ["foo", "bar"], "lval": [1, 2]})
right = pd.DataFrame({"key": ["foo", "bar"], "rval": [4, 5]})

In [16]:
left

,key,lval
0,foo,1
1,bar,2


In [17]:
right

,key,rval
0,foo,4
1,bar,5


- `on="key"` 키워드 인자
    - `key` 열에 사용된 항목별로 모든 경우의 수를 다룬다.
    - `foo` 값에 대해 `lval` 열에서 1개의 값이,
        `rval` 열에서 1개의 값이 있기에 `foo`와 관련해서 총 1개의 경우가 생성된다.
    - `bar` 값에 대해 `lval` 열에서 1개의 값이,
        `rval` 열에서 1개의 값이 있기에 `foo`와 관련해서 총 1개의 경우가 생성된다.
    <br><br>
    
    | `key` | `left.lval` | `right.rval` | 경우의 수 |
    | :---: | :---: | :---: | :---: |
    | `foo` | `1` | `4` | 1 |        
    | `bar` | `2` | `5` | 1 |        

In [18]:
pd.merge(left, right, on="key")

,key,lval,rval
0,foo,1,4
1,bar,2,5


**예제**

경우의 수는 지정된 열의 항목이 사용된 횟수를 기준으로 한다. 

In [19]:
left = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)

In [20]:
left

,key,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K2,A2,B2
3,K3,A3,B3


In [21]:
right = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)

In [22]:
right

,key,C,D
0,K0,C0,D0
1,K1,C1,D1
2,K2,C2,D2
3,K3,C3,D3


| `key` | (`left.A`, `left.B`) | (`right.C`, `right.D`) | 경우의 수 |
| :---: | :---: | :---: | :---: |
| `K0` | (`A0`, `B0`) | (`C0`, `D0`) | 1 |
| `K1` | (`A1`, `B1`) | (`C1`, `D1`) | 1 |
| `K2` | (`A2`, `B2`) | (`C2`, `D2`) | 1 |
| `K3` | (`A3`, `B3`) | (`C3`, `D3`) | 1 |

In [23]:
result = pd.merge(left, right, on="key")
result

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,C2,D2
3,K3,A3,B3,C3,D3


**다양한 키 활용**

- 두 개 이상의 키를 하나의 쌍으로 된 키를 사용하는 경우와 유사함. 

In [24]:
left = pd.DataFrame(
    {
        "key1": ["K0", "K0", "K1", "K2"],
        "key2": ["K0", "K1", "K0", "K1"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)

left

,key1,key2,A,B
0,K0,K0,A0,B0
1,K0,K1,A1,B1
2,K1,K0,A2,B2
3,K2,K1,A3,B3


In [25]:
right = pd.DataFrame(
    {
        "key1": ["K0", "K1", "K1", "K2"],
        "key2": ["K0", "K0", "K0", "K0"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)

right

,key1,key2,C,D
0,K0,K0,C0,D0
1,K1,K0,C1,D1
2,K1,K0,C2,D2
3,K2,K0,C3,D3


- `how='inner'`: 지정된 키의 교집합 대상

In [26]:
result = pd.merge(left, right, on=["key1", "key2"]) # how='inner' 가 기본값
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


In [27]:
result = pd.merge(left, right, how="inner", on=["key1", "key2"])
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


- `how='outer'`: 지정된 키의 합집합 대상

In [28]:
result = pd.merge(left, right, how="outer", on=["key1", "key2"])
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN
5,K2,K0,NaN,NaN,C3,D3


- `how='left'`: 왼쪽 데이터프레임의 키에 포함된 항목만 대상

In [29]:
left

,key1,key2,A,B
0,K0,K0,A0,B0
1,K0,K1,A1,B1
2,K1,K0,A2,B2
3,K2,K1,A3,B3


In [30]:
result = pd.merge(left, right, how="left", on=["key1", "key2"])
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN


- `how='right'`: 오른쪽 데이터프레임의 키에 포함된 항목만 대상

In [31]:
right

,key1,key2,C,D
0,K0,K0,C0,D0
1,K1,K0,C1,D1
2,K1,K0,C2,D2
3,K2,K0,C3,D3


In [32]:
result = pd.merge(left, right, how="right", on=["key1", "key2"])
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2
3,K2,K0,NaN,NaN,C3,D3


- `how='cross'`: 모든 경우의 수 조합

In [33]:
result = pd.merge(left, right, how="cross")
result

,key1_x,key2_x,A,B,key1_y,key2_y,C,D
0,K0,K0,A0,B0,K0,K0,C0,D0
1,K0,K0,A0,B0,K1,K0,C1,D1
2,K0,K0,A0,B0,K1,K0,C2,D2
3,K0,K0,A0,B0,K2,K0,C3,D3
4,K0,K1,A1,B1,K0,K0,C0,D0
5,K0,K1,A1,B1,K1,K0,C1,D1
6,K0,K1,A1,B1,K1,K0,C2,D2
7,K0,K1,A1,B1,K2,K0,C3,D3
8,K1,K0,A2,B2,K0,K0,C0,D0
9,K1,K0,A2,B2,K1,K0,C1,D1


### 합병: `DataFrame.join()` 메서드

인덱스를 기준으로 두 개의 데이터프레임을 합병할 때 사용한다.

In [34]:
left = pd.DataFrame(
    {"A": ["A0", "A1", "A2"], "B": ["B0", "B1", "B2"]}, index=["K0", "K1", "K2"]
)

left

,A,B
K0,A0,B0
K1,A1,B1
K2,A2,B2


In [35]:
right = pd.DataFrame(
    {"C": ["C0", "C2", "C3"], "D": ["D0", "D2", "D3"]}, index=["K0", "K2", "K3"]
)

right

,C,D
K0,C0,D0
K2,C2,D2
K3,C3,D3


In [36]:
left.join(right)

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2


아래와 같이 `pd.merge()` 함수를 이용한 결과와 동일하다.

In [37]:
pd.merge(left, right, left_index=True, right_index=True, how='left')

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2


`pd.merge()` 함수의 키워드 인자를 동일하게 사용할 수 있다.

- `how='outer'`

In [38]:
left.join(right, how="outer")

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2
K3,NaN,NaN,C3,D3


아래 코드가 동일한 결과를 낸다.

In [39]:
pd.merge(left, right, left_index=True, right_index=True, how='outer')

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2
K3,NaN,NaN,C3,D3


- `how='inner'`

In [40]:
left.join(right, how="inner")

,A,B,C,D
K0,A0,B0,C0,D0
K2,A2,B2,C2,D2


아래 코드가 동일한 결과를 낸다.

In [41]:
pd.merge(left, right, left_index=True, right_index=True, how='inner')

,A,B,C,D
K0,A0,B0,C0,D0
K2,A2,B2,C2,D2


## 다중 인덱스<font size='2'>MultiIndex</font>

- 참고: [Multiindex / advanced indexing](https://pandas.pydata.org/docs/user_guide/advanced.html)

다중 인덱스를 이용하여 데이터를 보다 체계적으로 다를 수 있다.
또한 이어서 다룰 그룹 분류<font size='2'>Group by</font>, 
모양 변환<font size='2'>reshaping</font>, 
피벗 변환<font size='2'>pivoting</font> 등에서 유용하게 활용된다.

### `MultiIndex` 객체

다중 인덱스 객체는 보통 튜플을 이용한다.
예를 들어 아래 두 개의 리스트를 이용하여 튜플을 생성한 다음 다중 인덱스로 만들어보자.

In [42]:
arrays = [
    ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
    ["one", "two", "one", "two", "one", "two", "one", "two"],
]

- 튜플 생성: 항목 8개

In [43]:
tuples = list(zip(*arrays))
tuples

[('bar', 'one'),
 ('bar', 'two'),
 ('baz', 'one'),
 ('baz', 'two'),
 ('foo', 'one'),
 ('foo', 'two'),
 ('qux', 'one'),
 ('qux', 'two')]

**다중 인덱스 객체 생성: `from_tupes()` 함수**

튜플 리스트를 이용하여 다중 인덱스 객체를 생성할 수 있다.

In [44]:
index = pd.MultiIndex.from_tuples(tuples)
index

MultiIndex([('bar', 'one'),
            ('bar', 'two'),
            ('baz', 'one'),
            ('baz', 'two'),
            ('foo', 'one'),
            ('foo', 'two'),
            ('qux', 'one'),
            ('qux', 'two')],
           )

- `names` 키워드 인자
    - 다중 인덱스의 각 레벨<font size='2'>level</font>의 이름 지정. 
    - 지정되지 않으면 `None`으로 처리됨.

예를 들어 위 코드에서 사용된 각각의 레벨에 이름은 다음과 같다.

- `"first"`: 0-레벨 이름
- `"second"`: 1-레벨 이름

In [45]:
index = pd.MultiIndex.from_tuples(tuples, names=["first", "second"])
index

MultiIndex([('bar', 'one'),
            ('bar', 'two'),
            ('baz', 'one'),
            ('baz', 'two'),
            ('foo', 'one'),
            ('foo', 'two'),
            ('qux', 'one'),
            ('qux', 'two')],
           names=['first', 'second'])

**다중 인덱스 객체 생성: `from_arrays()` 함수**

길이가 동일한 여러 개의 리스트로 구성된 어레이를 직접 이용할 수도 있다.

In [46]:
index = pd.MultiIndex.from_arrays(arrays, names=["first", "second"])
index

MultiIndex([('bar', 'one'),
            ('bar', 'two'),
            ('baz', 'one'),
            ('baz', 'two'),
            ('foo', 'one'),
            ('foo', 'two'),
            ('qux', 'one'),
            ('qux', 'two')],
           names=['first', 'second'])

### 다중 인덱스 라벨<font size='2'>label</font>을 사용하는 시리즈/데이터프레임 객체

- 시리즈 생성

아래 코드는 길이가 8인 어레이를 이용하여 시리즈를 생성한다.
인덱스의 라벨은 다중 인덱스가 사용된다.
각각의 레벨에서 라벨이 연속적으로 사용되는 경우는 보다 자연스러운 표현을 위해 생략되기도 한다.

In [47]:
s = pd.Series(np.random.randn(8), index=index)
s

first  second
bar    one       0.104651
       two       0.537733
baz    one       1.987001
       two       0.424737
foo    one      -0.796916
       two       1.369860
qux    one       0.232649
       two       0.089712
dtype: float64

- 데이터프레임 생성

아래 코드는 8개의 행으로 이뤄진 2차원 어레이를 이용하여 데이터프레임을 생성한다.
`index` 또는 `columns`로 여러 개의 리스트로 구성된 어레이를 지정하면
자동으로 다중 인덱스 라벨이 지정된다.

In [48]:
df = pd.DataFrame(np.random.randn(8, 4), index=arrays)
df

0         1         2         3
bar one  0.169186  1.283793 -1.790666  0.325007
    two -1.319538  0.888315  1.398552 -0.764955
baz one -2.293044 -0.120224 -0.080708 -0.442853
    two -1.889153  0.683859  1.073343 -0.419801
foo one -1.100325  0.035381  0.621623 -0.779864
    two  1.033033 -0.415391 -0.685320 -1.136405
qux one  0.158616  0.670204 -0.940846  0.184476
    two -1.880419  0.483080 -0.062087  0.671566

다중 인덱스를 열 라벨로도 활용할 수 있다.
아래 코드는 8개의 열로 이뤄진 2차원 어레이를 이용하여 데이터프레임을 생성한다.

In [49]:
df1 = pd.DataFrame(np.random.randn(3, 8), index=["A", "B", "C"], columns=index)
df1

first        bar                 baz                 foo                 qux  \
second       one       two       one       two       one       two       one   
A      -0.159286  0.224706  0.209860  0.801320  0.541192 -0.616051  0.866977   
B       0.618692  0.152904  0.771596  0.029026  0.586900  0.349560 -0.676974   
C      -1.051100  0.226440 -0.566494  1.127380  0.110879 -3.073640  0.658122   

first             
second       two  
A       0.209608  
B      -0.985949  
C       0.342262

인덱스 라벨과 열 라벨 모두 다중 인덱스를 이용할 수도 있다.

- 동일한 길이의 리스트로 이루어진 리스트를 인덱스 또는 열의 라벨로 지정하면
    다중 인덱스로 자동 지정된다.

In [50]:
arrays2 = [
    ["toto", "toto", "titi", "titi", "tata", "tata"],
    ["A", "B", "A", "B", "A", "B"],
]

In [51]:
pd.DataFrame(np.random.randn(6, 6), index=index[:6], columns=arrays2)

toto                titi                tata          
                     A         B         A         B         A         B
first second                                                            
bar   one     0.562591  0.160847  1.239032 -0.496532 -0.215398 -0.743129
      two     0.000575  0.352597 -1.790666 -0.673716 -0.297276  1.758015
baz   one    -1.068594  0.005278  1.319847  1.260190  0.178016 -0.472236
      two    -0.533751 -0.241971 -1.337261  0.095809 -0.610831 -0.676550
foo   one     1.208596 -1.957983 -1.721607 -0.421830 -1.043068  1.576159
      two    -0.243156 -0.153111  0.282822 -0.503019  0.516819  0.470666

**주의사항**

튜플을 라벨로 사용하는 것은 다중 인덱스와 아무 상관 없다.
단지 라벨이 튜플인 것 뿐이다.

In [52]:
tuples

[('bar', 'one'),
 ('bar', 'two'),
 ('baz', 'one'),
 ('baz', 'two'),
 ('foo', 'one'),
 ('foo', 'two'),
 ('qux', 'one'),
 ('qux', 'two')]

In [53]:
pd.Series(np.random.randn(8), index=tuples)

(bar, one)    1.063728
(bar, two)    0.781983
(baz, one)    0.333482
(baz, two)   -0.130083
(foo, one)   -0.006141
(foo, two)    0.869604
(qux, one)    0.791434
(qux, two)    0.312423
dtype: float64

### 인덱스의 레벨

다중 인덱스 객체의 `get_level_values()` 메서드를 이용하여 레벨별 인덱스 라벨을 확인할 수 있다.

- 0-레블 라벨

In [54]:
index.get_level_values(0)

Index(['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'], dtype='object', name='first')

레벨 이름을 이용할 수도 있다.

In [55]:
index.get_level_values("first")

Index(['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'], dtype='object', name='first')

- 1-레블 라벨

In [56]:
index.get_level_values(1)

Index(['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two'], dtype='object', name='second')

In [57]:
index.get_level_values("second")

Index(['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two'], dtype='object', name='second')

### 인덱싱

다중 인덱스를 라벨로 사용하는 시리즈와 데이터프레임의 인덱싱은 일반 인덱싱과 크게 다르지 않다.

- 시리즈 인덱싱

In [58]:
s

first  second
bar    one       0.104651
       two       0.537733
baz    one       1.987001
       two       0.424737
foo    one      -0.796916
       two       1.369860
qux    one       0.232649
       two       0.089712
dtype: float64

In [59]:
s["qux"]

second
one    0.232649
two    0.089712
dtype: float64

- 데이터프레임 인덱싱

In [60]:
df

0         1         2         3
bar one  0.169186  1.283793 -1.790666  0.325007
    two -1.319538  0.888315  1.398552 -0.764955
baz one -2.293044 -0.120224 -0.080708 -0.442853
    two -1.889153  0.683859  1.073343 -0.419801
foo one -1.100325  0.035381  0.621623 -0.779864
    two  1.033033 -0.415391 -0.685320 -1.136405
qux one  0.158616  0.670204 -0.940846  0.184476
    two -1.880419  0.483080 -0.062087  0.671566

In [61]:
df.loc["bar"]

,0,1,2,3
one,0.169186,1.283793,-1.790666,0.325007
two,-1.319538,0.888315,1.398552,-0.764955


레벨별로 라벨을 지정할 수 있다. 각각의 라벨은 쉼표로 구분한다.

In [62]:
df.loc["bar", "one"]

0    0.169186
1    1.283793
2   -1.790666
3    0.325007
Name: (bar, one), dtype: float64

아래와 같이 할 수도 있다.

In [63]:
df.loc["bar"].loc["one"]

0    0.169186
1    1.283793
2   -1.790666
3    0.325007
Name: one, dtype: float64

- 데이터프레임 인덱싱: 열 라벨이 다중 인덱스인 경우

In [64]:
df1

first        bar                 baz                 foo                 qux  \
second       one       two       one       two       one       two       one   
A      -0.159286  0.224706  0.209860  0.801320  0.541192 -0.616051  0.866977   
B       0.618692  0.152904  0.771596  0.029026  0.586900  0.349560 -0.676974   
C      -1.051100  0.226440 -0.566494  1.127380  0.110879 -3.073640  0.658122   

first             
second       two  
A       0.209608  
B      -0.985949  
C       0.342262

In [65]:
df1["bar"]

second,one,two
A,-0.159286,0.224706
B,0.618692,0.152904
C,-1.051100,0.226440


레벨별로 라벨을 지정한다. 각각의 라벨은 쉼표로 구분한다.

In [66]:
df1["bar", "one"]

A   -0.159286
B    0.618692
C   -1.051100
Name: (bar, one), dtype: float64

아래와 같이 할 수도 있다

In [67]:
df1["bar"]["one"]

A   -0.159286
B    0.618692
C   -1.051100
Name: one, dtype: float64

### 슬라이싱

다중 인덱스를 라벨로 사용하는 시리즈와 데이터프레임의 인덱싱은 일반 슬라이싱과 크게 다르지 않다.

In [68]:
df

0         1         2         3
bar one  0.169186  1.283793 -1.790666  0.325007
    two -1.319538  0.888315  1.398552 -0.764955
baz one -2.293044 -0.120224 -0.080708 -0.442853
    two -1.889153  0.683859  1.073343 -0.419801
foo one -1.100325  0.035381  0.621623 -0.779864
    two  1.033033 -0.415391 -0.685320 -1.136405
qux one  0.158616  0.670204 -0.940846  0.184476
    two -1.880419  0.483080 -0.062087  0.671566

- 0-레벨 인덱싱

In [69]:
df.loc["baz":"foo"]

0         1         2         3
baz one -2.293044 -0.120224 -0.080708 -0.442853
    two -1.889153  0.683859  1.073343 -0.419801
foo one -1.100325  0.035381  0.621623 -0.779864
    two  1.033033 -0.415391 -0.685320 -1.136405

- (0, 1)-레벨 인덱싱

In [70]:
df.loc[("baz", "two"):("qux", "one")]

0         1         2         3
baz two -1.889153  0.683859  1.073343 -0.419801
foo one -1.100325  0.035381  0.621623 -0.779864
    two  1.033033 -0.415391 -0.685320 -1.136405
qux one  0.158616  0.670204 -0.940846  0.184476

튜플들의 리스트를 지정하면 리인덱싱처럼 작동한다.

In [71]:
df.loc[[("bar", "two"), ("qux", "one")]]

,,0,1,2,3
bar,two,-1.319538,0.888315,1.398552,-0.764955
qux,one,0.158616,0.670204,-0.940846,0.184476


이외에 `slice()` 함수와 `pd.IndexSlice` 객체를 사용하는 방법도 있지만 여기서는 다루지 않는다.

## 그룹 분류: `pd.groupby()` 함수

- 참고: [Grouping section](https://pandas.pydata.org/docs/user_guide/groupby.html#groupby)

`pd.groupby()` 함수는 다음 3 기능을 제공한다.

- **분류**<font size='2'>Splitting</font>: 데이터를 조건에 따라 여러 그룹으로 분류
- **함수 적용**<font size='2'>Applying</font>: 그룹별로 함수 적용
- **조합**<font size='2'>Combining</font>: 그룹별 함수 적용 결과를 취합하여 새로운 데이터프레임/시리즈 생성

In [72]:
df = pd.DataFrame({'A': ['foo', 'bar', 'foo', 'bar',
                         'foo', 'bar', 'foo', 'bar'],
                   'B': ['one', 'one', 'two', 'three',
                         'two', 'two', 'one', 'three'],
                   'C': np.random.randn(8),
                   'D': np.random.randn(8)})

df

,A,B,C,D
0,foo,one,0.842391,0.203580
1,bar,one,-0.234987,-0.423858
2,foo,two,0.336531,1.264944
3,bar,three,-1.004781,-0.189630
4,foo,two,-1.133500,0.614676
5,bar,two,0.136441,-1.067721
6,foo,one,-0.419849,-1.548629
7,bar,three,0.360779,-0.717282


- `A` 열에 사용된 항목 기준으로 그룹으로 분류한 후 그룹별로 `C`와 `D` 열의 모든 항목의 합 계산해서 새로운 데이터프레임 생성
    <br><br>
    
    | `A` | 경우의 수 |
    | :---: | :---: |
    | `bar` | 1 |
    | `foo` | 1 |

In [73]:
df.groupby('A')[["C", "D"]].sum()

,C,D
A,,
bar,-0.742548,-2.398492
foo,-0.374428,0.534571


- `A`열의 항목과 `B` 열의 항목의 조합을 기준으로 그룹으로 그룹별로 `C`와 `D` 열의 모든 항목의 합 계산해서 새로운 데이터프레임 생성
    <br><br>
    
    | `A` | `B` | 경우의 수 |
    | :---: | :---: | :---: |
    | `bar` | `one`, `three`, `two` | 3 |
    | `foo` | `one`, `two` | 2 |

In [74]:
df.groupby(["A", "B"]).sum()

C         D
A   B                        
bar one   -0.234987 -0.423858
    three -0.644002 -0.906912
    two    0.136441 -1.067721
foo one    0.422541 -1.345050
    two   -0.796969  1.879620

**그룹 확인**

- `for` 반복문 활용 

In [75]:
for name, group in df.groupby(["A", "B"]):
    print(name)
    print(group)

('bar', 'one')
     A    B         C         D
1  bar  one -0.234987 -0.423858
('bar', 'three')
     A      B         C         D
3  bar  three -1.004781 -0.189630
7  bar  three  0.360779 -0.717282
('bar', 'two')
     A    B         C         D
5  bar  two  0.136441 -1.067721
('foo', 'one')
     A    B         C         D
0  foo  one  0.842391  0.203580
6  foo  one -0.419849 -1.548629
('foo', 'two')
     A    B         C         D
2  foo  two  0.336531  1.264944
4  foo  two -1.133500  0.614676


- `get_group()` 메서드

In [76]:
df.groupby(["A", "B"]).get_group(('bar', 'one'))

,A,B,C,D
1,bar,one,-0.234987,-0.423858


In [77]:
df.groupby(["A", "B"]).get_group(('bar', 'three'))

,A,B,C,D
3,bar,three,-1.004781,-0.189630
7,bar,three,0.360779,-0.717282


- `groups` 속성

In [78]:
df.groupby(["A", "B"]).groups

{('bar', 'one'): [1], ('bar', 'three'): [3, 7], ('bar', 'two'): [5], ('foo', 'one'): [0, 6], ('foo', 'two'): [2, 4]}

- `value_counts` 속성

In [79]:
df.groupby(["A", "B"]).value_counts()

A    B      C          D        
bar  one    -0.234987  -0.423858    1
     three  -1.004781  -0.189630    1
             0.360779  -0.717282    1
     two     0.136441  -1.067721    1
foo  one    -0.419849  -1.548629    1
             0.842391   0.203580    1
     two    -1.133500   0.614676    1
             0.336531   1.264944    1
dtype: int64

- `nunique` 속성

In [80]:
df.groupby(["A", "B"]).nunique()

C  D
A   B          
bar one    1  1
    three  2  2
    two    1  1
foo one    2  2
    two    2  2

- `sort=True` 키워드 인자

In [81]:
df.groupby(["A", "B"], sort=True).sum()

C         D
A   B                        
bar one   -0.234987 -0.423858
    three -0.644002 -0.906912
    two    0.136441 -1.067721
foo one    0.422541 -1.345050
    two   -0.796969  1.879620

In [82]:
df.groupby(["A", "B"], sort=False).sum()

C         D
A   B                        
foo one    0.422541 -1.345050
bar one   -0.234987 -0.423858
foo two   -0.796969  1.879620
bar three -0.644002 -0.906912
    two    0.136441 -1.067721

In [83]:
df.groupby(["A", "B"], sort=False).nunique()

C  D
A   B          
foo one    2  2
bar one    1  1
foo two    2  2
bar three  2  2
    two    1  1

**그룹 연산 예제**

- `max()` 메서드

In [84]:
df.groupby('A')[["C", "D"]].max()

,C,D
A,,
bar,0.360779,-0.189630
foo,0.842391,1.264944


In [85]:
df.groupby(["A", "B"]).max()

C         D
A   B                        
bar one   -0.234987 -0.423858
    three  0.360779 -0.189630
    two    0.136441 -1.067721
foo one    0.842391  0.203580
    two    0.336531  1.264944

- `mean()` 메서드

In [86]:
df.groupby('A')[["C", "D"]].mean()

,C,D
A,,
bar,-0.185637,-0.599623
foo,-0.093607,0.133643


In [87]:
df.groupby(["A", "B"]).mean()

C         D
A   B                        
bar one   -0.234987 -0.423858
    three -0.322001 -0.453456
    two    0.136441 -1.067721
foo one    0.211271 -0.672525
    two   -0.398485  0.939810

- `size()` 메서드

In [88]:
df.groupby('A')[["C", "D"]].size()

A
bar    4
foo    4
dtype: int64

In [89]:
df.groupby(["A", "B"]).size()

A    B    
bar  one      1
     three    2
     two      1
foo  one      2
     two      2
dtype: int64

- `describe()` 메서드

In [90]:
df.groupby('A')[["C", "D"]].describe()

C                                                              \
    count      mean       std       min       25%       50%       75%   
A                                                                       
bar   4.0 -0.185637  0.598815 -1.004781 -0.427436 -0.049273  0.192526   
foo   4.0 -0.093607  0.865818 -1.133500 -0.598262 -0.041659  0.462996   

                  D                                                    \
          max count      mean       std       min       25%       50%   
A                                                                       
bar  0.360779   4.0 -0.599623  0.379450 -1.067721 -0.804892 -0.570570   
foo  0.842391   4.0  0.133643  1.203629 -1.548629 -0.234473  0.409128   

                         
          75%       max  
A                        
bar -0.365301 -0.189630  
foo  0.777243  1.264944

In [91]:
df.groupby(["A", "B"]).describe()

C                                                              \
          count      mean       std       min       25%       50%       75%   
A   B                                                                         
bar one     1.0 -0.234987       NaN -0.234987 -0.234987 -0.234987 -0.234987   
    three   2.0 -0.322001  0.965597 -1.004781 -0.663391 -0.322001  0.019389   
    two     1.0  0.136441       NaN  0.136441  0.136441  0.136441  0.136441   
foo one     2.0  0.211271  0.892539 -0.419849 -0.104289  0.211271  0.526831   
    two     2.0 -0.398485  1.039469 -1.133500 -0.765992 -0.398485 -0.030977   

                        D                                                    \
                max count      mean       std       min       25%       50%   
A   B                                                                         
bar one   -0.234987   1.0 -0.423858       NaN -0.423858 -0.423858 -0.423858   
    three  0.360779   2.0 -0.453456  0.373106 -0.717282 -0.585369 -0.453456   
    two    0.136441   1.0 -1.067721       NaN -1.067721 -1.067721 -1.067721   
foo one    0.842391   2.0 -0.672525  1.238999 -1.548629 -1.110577 -0.672525   
    two    0.336531   2.0  0.939810  0.459809  0.614676  0.777243  0.939810   

                               
                75%       max  
A   B                          
bar one   -0.423858 -0.423858  
    three -0.321543 -0.189630  
    two   -1.067721 -1.067721  
foo one   -0.234473  0.203580  
    two    1.102377  1.264944

## 모양 변환<font size='2'>Reshaping</font>

- 참고: [Reshaping](https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-stacking)

### 스택

열 인덱스의 레벨을 하나 줄일 때 사용한다.
없어진 레벨은 행 인덱스의 마지막 레벨로 추가된다.

In [92]:
index

MultiIndex([('bar', 'one'),
            ('bar', 'two'),
            ('baz', 'one'),
            ('baz', 'two'),
            ('foo', 'one'),
            ('foo', 'two'),
            ('qux', 'one'),
            ('qux', 'two')],
           names=['first', 'second'])

In [93]:
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=["A", "B"])
df

A         B
first second                    
bar   one     0.800029 -0.970989
      two     0.397594  1.547701
baz   one     0.985661 -0.960152
      two     1.248146  0.518137
foo   one    -0.118944 -2.875428
      two    -0.540819  1.816586
qux   one     1.151483 -1.821522
      two     1.906664 -0.012671

In [94]:
df2 = df[:4]
df2

A         B
first second                    
bar   one     0.800029 -0.970989
      two     0.397594  1.547701
baz   one     0.985661 -0.960152
      two     1.248146  0.518137

- `stack()` 메서드: 
    열이 한 개의 레벨로 구성되어 있기에 `stack()` 메서드를 적용하면
    결국 모든 열이 없어지고, 열의 라벨은
    인덱스의 마지막 레벨의 라벨로 변환된다.
    여기서는 결국 3중 인덱스를 사용하는 시리즈를 생성한다.

In [95]:
stacked = df2.stack()
stacked

first  second   
bar    one     A    0.800029
               B   -0.970989
       two     A    0.397594
               B    1.547701
baz    one     A    0.985661
               B   -0.960152
       two     A    1.248146
               B    0.518137
dtype: float64

### 언스택

행 인덱스의 지정된 레벨을 열의 마지막 레벨로 변환한다.
인자를 지정하지 않으면 마지막 레벨을 변환한다.

- `unstack()` 메서드

In [96]:
stacked.unstack()

A         B
first second                    
bar   one     0.800029 -0.970989
      two     0.397594  1.547701
baz   one     0.985661 -0.960152
      two     1.248146  0.518137

In [97]:
stacked.unstack().unstack()

A                   B          
second       one       two       one       two
first                                         
bar     0.800029  0.397594 -0.970989  1.547701
baz     0.985661  1.248146 -0.960152  0.518137

인자를 지정하면 해당 레벨을 열의 마지막 레벨로 변환한다.

In [98]:
stacked.unstack(0)

first          bar       baz
second                      
one    A  0.800029  0.985661
       B -0.970989 -0.960152
two    A  0.397594  1.248146
       B  1.547701  0.518137

In [99]:
stacked.unstack(1)

second        one       two
first                      
bar   A  0.800029  0.397594
      B -0.970989  1.547701
baz   A  0.985661  1.248146
      B -0.960152  0.518137

### 피버팅<font size='2'>Pivoting</font>

- 참고: [Pivot Tables](https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-pivot)

**`pd.pivot_table()` 함수**

**예제**

In [100]:
import datetime

df = pd.DataFrame(
    {
        "A": ["one", "one", "two", "three"] * 6,
        "B": ["A", "B", "C"] * 8,
        "C": ["foo", "foo", "foo", "bar", "bar", "bar"] * 4,
        "D": np.random.randn(24),
        "E": np.random.randn(24),
    }
)

df

,A,B,C,D,E
0,one,A,foo,1.418453,0.290003
1,one,B,foo,0.173762,-0.131812
2,two,C,foo,0.138150,-0.365551
3,three,A,bar,-0.191298,-1.729415
4,one,B,bar,-0.012960,-1.279552
5,one,C,bar,-1.464918,-0.791492
6,two,A,foo,-0.452366,0.018174
7,three,B,foo,0.296954,0.428367
8,one,C,foo,0.618624,-0.099844
9,one,A,bar,0.516960,1.208930


In [101]:
pd.pivot_table(df, values="D", index=["A", "B"], columns=["C"]) # aggfunc=np.mean 이 기본값

C             bar       foo
A     B                    
one   A  1.229201  0.996160
      B  0.592906  0.038530
      C -1.823279 -0.034685
three A -0.190459       NaN
      B       NaN  1.250215
      C  0.000199       NaN
two   A       NaN -0.189818
      B -0.696446       NaN
      C       NaN  0.012498

In [102]:
pd.pivot_table(df, values="D", index=["A", "B"], columns=["C"], aggfunc=np.sum)

C             bar       foo
A     B                    
one   A  2.458403  1.992320
      B  1.185812  0.077059
      C -3.646558 -0.069371
three A -0.380918       NaN
      B       NaN  2.500430
      C  0.000399       NaN
two   A       NaN -0.379637
      B -1.392893       NaN
      C       NaN  0.024997

**`DataFrame.pivot()` 메서드**

In [103]:
df

,A,B,C,D,E
0,one,A,foo,1.418453,0.290003
1,one,B,foo,0.173762,-0.131812
2,two,C,foo,0.138150,-0.365551
3,three,A,bar,-0.191298,-1.729415
4,one,B,bar,-0.012960,-1.279552
5,one,C,bar,-1.464918,-0.791492
6,two,A,foo,-0.452366,0.018174
7,three,B,foo,0.296954,0.428367
8,one,C,foo,0.618624,-0.099844
9,one,A,bar,0.516960,1.208930


In [104]:
df1 = df.groupby(['A', 'B', 'C']).sum().reset_index()
df1

,A,B,C,D,E
0,one,A,bar,2.458403,2.027823
1,one,A,foo,1.992320,0.878339
2,one,B,bar,1.185812,-1.847242
3,one,B,foo,0.077059,1.035692
4,one,C,bar,-3.646558,-1.857255
5,one,C,foo,-0.069371,-1.092065
6,three,A,bar,-0.380918,-0.663987
7,three,B,foo,2.500430,0.909458
8,three,C,bar,0.000399,-1.801149
9,two,A,foo,-0.379637,-1.236296


In [105]:
df1.pivot(index=['A', 'B'], columns='C', values="D")

C             bar       foo
A     B                    
one   A  2.458403  1.992320
      B  1.185812  0.077059
      C -3.646558 -0.069371
three A -0.380918       NaN
      B       NaN  2.500430
      C  0.000399       NaN
two   A       NaN -0.379637
      B -1.392893       NaN
      C       NaN  0.024997

## 시계열<font size='2'>time series</font>

시계열 데이터는 시간의 흐름을 다루는 데이터다. 증시, 날씨 데이터 등에서 많이 사용된다.
판다스 라이브러리는 시계열 데이터를 다루는 많은 기능을 제공한다.

- 참고: [Time Series](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries)

**`pd.date_range()` 함수**

시간을 기준으로 시간 인덱스 자료형을 생성한다.
아래 코드에 사용된 키워드 인자의 의미는 다음과 같다.

- `periods=100`: 첫째 인자로 지정된 시간부터 100개의 시간 데이터 샘플 생성
- `freq="S"`: 100개의 시간 데이터 샘플을 초 단위로 생성

In [106]:
rng = pd.date_range("1/1/2023", periods=100, freq="S")
rng

DatetimeIndex(['2023-01-01 00:00:00', '2023-01-01 00:00:01',
               '2023-01-01 00:00:02', '2023-01-01 00:00:03',
               '2023-01-01 00:00:04', '2023-01-01 00:00:05',
               '2023-01-01 00:00:06', '2023-01-01 00:00:07',
               '2023-01-01 00:00:08', '2023-01-01 00:00:09',
               '2023-01-01 00:00:10', '2023-01-01 00:00:11',
               '2023-01-01 00:00:12', '2023-01-01 00:00:13',
               '2023-01-01 00:00:14', '2023-01-01 00:00:15',
               '2023-01-01 00:00:16', '2023-01-01 00:00:17',
               '2023-01-01 00:00:18', '2023-01-01 00:00:19',
               '2023-01-01 00:00:20', '2023-01-01 00:00:21',
               '2023-01-01 00:00:22', '2023-01-01 00:00:23',
               '2023-01-01 00:00:24', '2023-01-01 00:00:25',
               '2023-01-01 00:00:26', '2023-01-01 00:00:27',
               '2023-01-01 00:00:28', '2023-01-01 00:00:29',
               '2023-01-01 00:00:30', '2023-01-01 00:00:31',
               '2023-01-

아래 코드는 0부터 500 사이에서 100개의 정수를 임의로 생성한 다음에 
위 시간 인덱스를 이용하여 시리즈를 생성한다.

In [107]:
ts = pd.Series(np.random.randint(0, 500, len(rng)), index=rng)
ts

2023-01-01 00:00:00    281
2023-01-01 00:00:01    330
2023-01-01 00:00:02    325
2023-01-01 00:00:03    382
2023-01-01 00:00:04    325
                      ... 
2023-01-01 00:01:35    432
2023-01-01 00:01:36    167
2023-01-01 00:01:37    463
2023-01-01 00:01:38      7
2023-01-01 00:01:39    176
Freq: S, Length: 100, dtype: int64

**`resample()` 메서드**

 인자로 지정된 기준으로 구룹 생성 후 집계<font size='2'>aggregation</font> 실행한다.

아래 코드의 의미는 다음과 같다.
- `"10S"`: 10초 단위로 데이터를 그룹으로 묶기
- `sum()` 메서도: 그룹별로 속한 데이터 샘플 모두 더하기

In [108]:
ts.resample("10S").sum()

2023-01-01 00:00:00    3141
2023-01-01 00:00:10    3111
2023-01-01 00:00:20    1957
2023-01-01 00:00:30    3057
2023-01-01 00:00:40    2099
2023-01-01 00:00:50    2094
2023-01-01 00:01:00    2092
2023-01-01 00:01:10    2608
2023-01-01 00:01:20    2446
2023-01-01 00:01:30    2682
Freq: 10S, dtype: int64

아래 코드의 의미는 다음과 같다.
- `"1Min"`: 1분 단위로 데이터를 그룹으로 묶기
- `sum()` 메서도: 그룹별로 속한 데이터 샘플 모두 더하기

In [109]:
ts.resample("1Min").sum()

2023-01-01 00:00:00    15459
2023-01-01 00:01:00     9828
Freq: T, dtype: int64

**시간 대 기간**

시간<font size='2'>time stamp</font>과 기간<font size='2'>period</font>의 차이점과
상호 변환 방식을 살펴본다.

- 2023년 1월 1일부터 월 단위로 12개의 항목을 갖는 시간 인덱스 생성

In [110]:
rng = pd.date_range("1/1/2023", periods=12, freq="M")
rng

DatetimeIndex(['2023-01-31', '2023-02-28', '2023-03-31', '2023-04-30',
               '2023-05-31', '2023-06-30', '2023-07-31', '2023-08-31',
               '2023-09-30', '2023-10-31', '2023-11-30', '2023-12-31'],
              dtype='datetime64[ns]', freq='M')

- 앞서 생성된 시간 인덱스를 라벨로 사용하는 시리즈 생성

In [111]:
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

2023-01-31   -0.419162
2023-02-28   -1.429591
2023-03-31   -1.107081
2023-04-30    0.138551
2023-05-31   -0.566984
2023-06-30   -1.838061
2023-07-31   -0.735645
2023-08-31    0.288954
2023-09-30   -0.142278
2023-10-31    0.692142
2023-11-30   -0.083723
2023-12-31    2.334320
Freq: M, dtype: float64

**`to_period() 메서드`**

- 시간 인덱스를 월 단위의 기간 인덱스로 변환

In [112]:
ps = ts.to_period()
ps

2023-01   -0.419162
2023-02   -1.429591
2023-03   -1.107081
2023-04    0.138551
2023-05   -0.566984
2023-06   -1.838061
2023-07   -0.735645
2023-08    0.288954
2023-09   -0.142278
2023-10    0.692142
2023-11   -0.083723
2023-12    2.334320
Freq: M, dtype: float64

**`to_timestamp() 메서드`**

- 기간 인덱스를 월 단위의 시간 인덱스로 변환

In [113]:
ps.to_timestamp()

2023-01-01   -0.419162
2023-02-01   -1.429591
2023-03-01   -1.107081
2023-04-01    0.138551
2023-05-01   -0.566984
2023-06-01   -1.838061
2023-07-01   -0.735645
2023-08-01    0.288954
2023-09-01   -0.142278
2023-10-01    0.692142
2023-11-01   -0.083723
2023-12-01    2.334320
Freq: MS, dtype: float64

## 데이터셋 불러오기와 저장하기

**csv 파일 불러오기**

- 참고: [`DataFrame.to_csv()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html#pandas.DataFrame.to_csv)

In [114]:
df.to_csv("foo.csv")

**csv 파일로 저장하기**

- 참고: [`read_csv()`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html#pandas.read_csv)

In [115]:
pd.read_csv("foo.csv")

,Unnamed: 0,A,B,C,D,E
0,0,one,A,foo,1.418453,0.290003
1,1,one,B,foo,0.173762,-0.131812
2,2,two,C,foo,0.138150,-0.365551
3,3,three,A,bar,-0.191298,-1.729415
4,4,one,B,bar,-0.012960,-1.279552
5,5,one,C,bar,-1.464918,-0.791492
6,6,two,A,foo,-0.452366,0.018174
7,7,three,B,foo,0.296954,0.428367
8,8,one,C,foo,0.618624,-0.099844
9,9,one,A,bar,0.516960,1.208930


**엑셀 파일 불러오기**

- 참고: [`DataFrame.to_excel()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html#pandas.DataFrame.to_excel)

In [116]:
df.to_excel("foo.xlsx", sheet_name="Sheet1")

**엑셀 파일로 저장하기**

- 참고: [`read_excel()`](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html#pandas.read_excel)

In [117]:
pd.read_excel("foo.xlsx", "Sheet1", index_col=None, na_values=["NA"])

,Unnamed: 0,A,B,C,D,E
0,0,one,A,foo,1.418453,0.290003
1,1,one,B,foo,0.173762,-0.131812
2,2,two,C,foo,0.138150,-0.365551
3,3,three,A,bar,-0.191298,-1.729415
4,4,one,B,bar,-0.012960,-1.279552
5,5,one,C,bar,-1.464918,-0.791492
6,6,two,A,foo,-0.452366,0.018174
7,7,three,B,foo,0.296954,0.428367
8,8,one,C,foo,0.618624,-0.099844
9,9,one,A,bar,0.516960,1.208930
